In [ ]:
# Cell 1: imports & plotting setup

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Basic plotting style (NMI-like: clean, no top/right spines)
plt.rcParams["figure.dpi"] = 150
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.alpha"] = 0.3
plt.rcParams["font.size"] = 9
plt.rcParams["legend.frameon"] = False

tables_dir = os.path.join("..", "results", "tables")
csv_path = os.path.join(tables_dir, "exp11_shape_texture_pier.csv")
df = pd.read_csv(csv_path)

df.head()

In [ ]:
# Cell 2: small helper to enforce a consistent model order and colors

model_order = [
    "ResNet18",
    "ResNet50",
    "EfficientNetB0",
    "ConvNeXtTiny",
    "ViT_B16",
    "ShapeResNet50_SIN",
]

color_map = {
    "ResNet18": "tab:blue",
    "ResNet50": "tab:orange",
    "EfficientNetB0": "tab:green",
    "ConvNeXtTiny": "tab:red",
    "ViT_B16": "tab:purple",
    "ShapeResNet50_SIN": "tab:brown",
}

def order_models(sub_df: pd.DataFrame) -> pd.DataFrame:
    sub = sub_df.copy()
    sub["Order"] = sub["TargetModel"].map({m: i for i, m in enumerate(model_order)})
    sub = sub.sort_values("Order")
    sub = sub.drop(columns=["Order"])
    return sub


In [ ]:
# Cell 3: aggregate mean PIER per context and model

agg = (
    df.groupby(["ContextLabel", "TargetModel"])["MeanPIER"]
    .mean()
    .reset_index()
)

texture_df = order_models(agg[agg["ContextLabel"] == "texture_natural"])
shape_df   = order_models(agg[agg["ContextLabel"] == "shape_bias"])

texture_df, shape_df


In [ ]:
# Cell 4: barplots for texture vs shape contexts (two panels)

fig, axes = plt.subplots(1, 2, figsize=(8, 3.0), sharey=True)

for ax, (sub_df, title) in zip(
    axes,
    [
        (texture_df, "Texture‑dominated context (natural images)"),
        (shape_df,   "Shape‑dominated context (cue‑conflict / stylized)"),
    ],
):
    models = sub_df["TargetModel"].tolist()
    vals = sub_df["MeanPIER"].values
    x = np.arange(len(models))
    colors = [color_map[m] for m in models]

    ax.bar(x, vals, color=colors)
    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=45, ha="right")
    ax.set_ylabel("Mean PIER")
    ax.set_title(title, fontsize=10)

    for xi, v in zip(x, vals):
        ax.text(xi, v, f"{v:.2f}", ha="center", va="bottom", fontsize=7)

    ax.grid(axis="y", alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Cell 5: delta PIER (shape context minus texture context) per model

# Merge the two contexts into a wide table
wide = texture_df[["TargetModel", "MeanPIER"]].rename(
    columns={"MeanPIER": "PIER_Texture"}
).merge(
    shape_df[["TargetModel", "MeanPIER"]].rename(
        columns={"MeanPIER": "PIER_Shape"}
    ),
    on="TargetModel",
    how="inner",
)

wide["Delta_PIER"] = wide["PIER_Shape"] - wide["PIER_Texture"]
wide = order_models(wide)

wide


In [ ]:
# Cell 6: delta barplot (one panel)

models = wide["TargetModel"].tolist()
vals = wide["Delta_PIER"].values
x = np.arange(len(models))
colors = [color_map[m] for m in models]

plt.figure(figsize=(4.5, 3.0))
plt.bar(x, vals, color=colors)
plt.axhline(0.0, color="black", linewidth=0.8, linestyle="--")

plt.xticks(x, models, rotation=45, ha="right")
plt.ylabel("Δ PIER = PIER_shape − PIER_texture")
plt.title("Change in uniqueness from texture to shape context")

for xi, v in zip(x, vals):
    plt.text(xi, v, f"{v:.2f}", ha="center", va="bottom", fontsize=7)

plt.tight_layout()
plt.show()
